In [50]:
##1.import library 還有導入美國公司的查詢代碼excel表
###需要載入公司代碼excel，桌面要有資料夾，之後會輸出excel(公司資訊，狀況)，以及docx(爬下來的內文)
###美國公司年報在某個年分之後就沒有item 1A 所以蠻多會找不到，而有些公司則是沒有10k檔案

In [31]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import docx #word檔
import os #寫入新的directory
import time #timesleep

from openpyxl import load_workbook
from openpyxl import Workbook
import xlrd
import xlwt
from xlrd import open_workbook

#獲取公司cik的excel
filename = r'C:\Users\wii76_000\Desktop\cik_allfirm2.xlsx'
book = xlrd.open_workbook(filename)
allcik = []
# 獲取工作表的方法之一，用下標。
sheel_1 = book.sheet_by_index(1)
# 打印第一個工作表的名的方法之一。
print("Worksheet name(s): ",book.sheet_names()[1])
print('sheel_1.name:',sheel_1.name,'sheel_1.nrows:',sheel_1.nrows,'sheel_1.ncols:',sheel_1.ncols)
for i in range(sheel_1.nrows):
    allcik.append(str(sheel_1.cell_value(i,colx=0)))
 

Worksheet name(s):  Final
sheel_1.name: Final sheel_1.nrows: 100 sheel_1.ncols: 6


In [ ]:
###這裡查找item 1 1A 7的方式不算是最好，還是有時候會沒找到該部分，還有修改的空間

In [36]:
#持續寫入資料的excel
wb = Workbook()
#選取正在工作中的表單
ws = wb.active
cnt = 
errnocom = []
errno10k = []

for i in allcik:
    cnt += 1
    print("************************第",cnt,"筆資料*************************")
    i = i.rstrip('0').rstrip('.')

    payload={
        'action':'getcompany',
        'CIK':i,
        'type':'10-K',
        'owner':'exclude',
        'count':'40'
    }

    url='https://www.sec.gov/cgi-bin/browse-edgar'
    a=requests.get(url,params=payload)
    soup =BeautifulSoup(a.text,"html.parser")

    #找公司名稱、CIK
    cname = []
    ccik = ""
    e = soup.find("span",{"class" : {"companyName"}})
    if(soup.find("span",{"class" : {"companyName"}}) == None):
        print("Something is missing, the order num:",cnt)
        errnum.append(cnt)
        continue
    for text in e.stripped_strings:
        #print(text)
        cname.append(text)

    ccik = int(str("".join(re.findall("[0-9]",cname[3]))))
    print("CompanyName :",cname[0])
    print("CompanyCik :",ccik)

    #找第一層的URL 用a標籤找href
    b = soup.select("a")
    href=[]
    docurl=[]
    for i in range(len(b)):
        if(b[i]["href"][1:4]=='Arc'):
            temp=b[i]["href"]
            href.append(temp)
    
    ct = 0
    lastyear = 0
    url3 = ""
    cform = []
    form = ""
    tkurl = []
    year = 0
    filDate = ""
    perReport = ""

    #同上一步作法 再次用a標籤找href
    for j in href:
        url2='https://www.sec.gov'+j
        layer2 = requests.get(url2)
        soup2 =BeautifulSoup(layer2.text,"html.parser")
        c = soup2.select("a")
        d = soup2.select("td")
        f = soup2.find_all("div",{"class" : {"infoHead","info"}})
        h = soup2.find("div",{"id" : {"formName"}})
        print("h",h)
        for text in h.stripped_strings:
            form = text
            break
        if(form == "Form 10-K" or form == "Form 10-K405"):
            print(url2)
            print(form)
            j = soup2.find_all("td",{"scope" : {"row"}})

            filDate = f[1].text
            perReport = f[7].text
            print(f[7].text[0:4])
            year = int(f[7].text[0:4])
            month = int(f[7].text[5:7])
            day = int(f[7].text[8:10])

            if(".htm" in j[2].find("a")["href"]):
                urlast = 'https://www.sec.gov'+j[2].find("a")["href"]
            elif(".tx" in j[2].find("a")["href"]):
                urlast = 'https://www.sec.gov'+j[2].find("a")["href"]
            else:
                #在completesubmission的txt檔
                for k in range(len(j)):
                    if("Complete" in j[k].text):
                        #print("Complete submission text file",j[k+1].text)
                        #print("10K或10K405文檔",'https://www.sec.gov'+j[k+1].find("a")["href"])
                        urlast = 'https://www.sec.gov'+j[k+1].find("a")["href"]
                        break
            #判斷報告年分
            if(month<6):
                year = year - 1
            elif(month == 6 and day <= 16):year = year - 1          
            lastyear = year
            #tkurl.append([year,urlast])
            
    #print(tkurl)
            #進入到10k檔案，決定是txt還是html，txt直接用文字搜尋，html就用標籤抓
            #初始化
            al = ""
            state = []
            start1 = 0
            end1 = 0
            end1to2 = 0

            start1A = 0
            end1A = 0

            start7 = 0
            end7 = 0
            end7to8 = 0
            
            #10k檔案解析
            para = requests.get(urlast)
            soup3 = BeautifulSoup(para.text,"html.parser")
            article = soup3.text

            if(re.search("Item 1\.",article, flags = re.IGNORECASE)):
                start1 = re.search("Item 1\.",article, flags = re.IGNORECASE).span()[0]
            if(re.search("Item 1A\.",article, flags = re.IGNORECASE)):
                end1 = re.search("Item 1A\.",article, flags = re.IGNORECASE).span()[0]
                start1A = end1
            if(re.search("Item 1B\.",article, flags = re.IGNORECASE)):
                end1A = re.search("Item 1B\.",article, flags = re.IGNORECASE).span()[0]
            elif(re.search("Item 2\.",article, flags = re.IGNORECASE)):
                end1to2 = re.search("Item 2\.",article, flags = re.IGNORECASE).span()[0]

            if(re.search("Item 7\.",article, flags = re.IGNORECASE)):
                start7 = re.search("Item 7\.",article, flags = re.IGNORECASE).span()[0]
            if(re.search("Item 7A\.",article, flags = re.IGNORECASE)):
                end7 = re.search("Item 7A\.",article, flags = re.IGNORECASE).span()[0]
            elif(re.search("Item 8\.",article, flags = re.IGNORECASE)):
                end7to8 = re.search("Item 8\.",article, flags = re.IGNORECASE).span()[0]

            print(start1,end1,end1to2)
            print(start1A,end1A)
            print(start7,end7,end7to8)

            state = [str(ccik),str(ccik)+"_"+str(year)+".docx",filDate,perReport,str(year), urlast,str(start1) ,str(end1),
                     str(end1to2),str(start1A), str(end1A), str(start7), str(end7) , str(end7to8)]

            #判斷是否有找到該檔案
            if(start1 != 0 and end1 != 0):
                al += article[start1:end1]+"\n*****************************************\n"
            elif(start1 != 0 and end1to2 != 0):  
                al += article[start1:end1to2]+"\n*****************************************\n"
                state.append("沒有item 1A，item1結尾到item2")
                print("沒有item 1A，item1結尾到item2")
            else:
                state.append("沒有item 1")
                print("沒有item 1")

            if(start1A !=0):
                al += article[start1A:end1A]+"\n*****************************************\n"

            if(start7 != 0 and end7 != 0):
                al += article[start7:end7]+"\n*****************************************\n"
            elif(start7 != 0 and end7to8 != 0): 
                al += article[start7:end7to8]+"\n*****************************************\n" 
                state.append("沒有item 7A，item7結尾到item8")
                print("沒有item 7A，item7結尾到item8")
            else:
                state.append("沒有item 7")
                print("沒有item 7")
                
            print(state)
            
            #創建檔案放置資料夾
            newdir = r'C:\Users\wii76_000\Desktop\skill_labor' + '\\' + str(ccik)
            #判斷資料夾是否存在
            if not os.path.exists(newdir):
                #建立資料夾
                os.makedirs(newdir)

            #寫入10kword檔
            doc = docx.Document()
            doc.add_paragraph(al)
            doc.save(newdir+'\\'+str(ccik)+"_"+str(year)+".docx")
            
            # 向下新增一列並連續插入值
            ws.append(state)
            wb.save(r'C:\Users\wii76_000\Desktop\allinfo.xlsx')
            print("errnocom",errnocom)
        elif("10-K" not in form):
            errno10k.append(cnt)
            # 向下新增一列並連續插入值
            ws.append([ccik,"No 10k file"])
            wb.save(r'C:\Users\wii76_000\Desktop\allinfo.xlsx')
            print("有該家公司，但沒有10k檔案")
        
        

************************第 1 筆資料*************************
20
CompanyName : K TRON INTERNATIONAL INC
CompanyCik : 20
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/20/000095012310024631/0000950123-10-024631-index.htm
Form 10-K
2010
0 0 0
0 0
0 0 0
沒有item 1
沒有item 7
['20', '20_2009.docx', '2010-03-15', '2010-01-02', '2009', 'https://www.sec.gov/Archives/edgar/data/20/000095012310024631/c97665e10vk.htm', '0', '0', '0', '0', '0', '0', '0', '0', '沒有item 1', '沒有item 7']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/20/000089322009000558/0000893220-09-000558-index.htm
Form 10-K
2009
0 0 0
0 0
0 0 0
沒有item 1
沒有item 7
['20', '20_2008.docx', '2009-03-13', '2009-01-03', '2008', 'https://www.sec.gov/Archives/edgar/data/20/000089322009000558/w72345e10vk.htm', '0'

6033 25886 0
25886 53051
73967 118993 0
['1750', '1750_2016.docx', '2017-07-12', '2017-05-31', '2016', 'https://www.sec.gov/Archives/edgar/data/1750/000104746917004528/a2232622z10-k.htm', '6033', '25886', '0', '25886', '53051', '73967', '118993', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1750/000104746916014299/0001047469-16-014299-index.htm
Form 10-K
2016
5601 23359 0
23359 49966
69052 116865 0
['1750', '1750_2015.docx', '2016-07-13', '2016-05-31', '2015', 'https://www.sec.gov/Archives/edgar/data/1750/000104746916014299/a2228768z10-k.htm', '5601', '23359', '0', '23359', '49966', '69052', '116865', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1750/000104746915006136/0001047469-15-006136-index.htm
Form 10-K
2015
0 25805 0


h <div id="formName">
<strong>Form 10-K405</strong> - Annual report [Sections 13 and 15(d), S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1750/000091205700039006/0000912057-00-039006-index.htm
Form 10-K405
2000
3834 0 3927
0 0
4565 4739 0
沒有item 1A，item1結尾到item2
['1750', '1750_1999.docx', '2000-08-24', '2000-05-31', '1999', 'https://www.sec.gov/Archives/edgar/data/1750/000091205700039006/a10-k405.txt', '3834', '0', '3927', '0', '0', '4565', '4739', '0', '沒有item 1A，item1結尾到item2']
errnocom []
h <div id="formName">
<strong>Form 10-K405</strong> - Annual report [Sections 13 and 15(d), S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1750/0001047469-99-032978-index.html
Form 10-K405
1999
5001 0 5115
0 0
5905 6108 0
沒有item 1A，item1結尾到item2
['1750', '1750_1998.docx', '1999-08-18', '1999-05-31', '1998', 'https://www.sec.gov/Archives/edgar/data/1750/000104746999032978/0001047469-99-032978.txt', '5001', '0', '5115', '0', '0', '5905', '6108', '0', '沒有it

errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1800/000104746914001176/0001047469-14-001176-index.htm
Form 10-K
2013
3523 29387 0
29387 54065
77668 148573 0
['1800', '1800_2013.docx', '2014-02-21', '2013-12-31', '2013', 'https://www.sec.gov/Archives/edgar/data/1800/000104746914001176/a2218043z10-k.htm', '3523', '29387', '0', '29387', '54065', '77668', '148573', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K/A</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: <b>[Amend]</b>
</div>
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1800/000104746913001180/0001047469-13-001180-index.htm
Form 10-K
2012
3522 34993 0
34993 60155
86321 172433 0
['1800', '1800_2012.docx', '2013-02-15', '2012-12-31', '2012', 'https://www.sec.gov/Archive

h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1800/0001047469-98-009108-index.html
Form 10-K
1997
4557 0 30756
0 0
61429 61776 0
沒有item 1A，item1結尾到item2
['1800', '1800_1997.docx', '1998-03-09', '1997-12-31', '1997', 'https://www.sec.gov/Archives/edgar/data/1800/0001047469-98-009108.txt', '4557', '0', '30756', '0', '0', '61429', '61776', '0', '沒有item 1A，item1結尾到item2']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/1800/0000912057-97-008391-index.html
Form 10-K
1996
4525 0 29867
0 0
56052 0 56399
沒有item 1A，item1結尾到item2
沒有item 7A，item7結尾到item8
['1800', '1800_1996.docx', '1997-03-11', '1996-12-31', '1996', 'https://www.sec.gov/Archives/edgar/data/1800/0000912057-97-008391.txt', '4525', '0', '29867', '0', '0', '56052', '0', '56399', '沒有item 1A，item1結尾到

...

errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4962/000119312506046095/0001193125-06-046095-index.htm
Form 10-K
2005
0 7571 0
7571 0
0 0 0
沒有item 1
沒有item 7
['4962', '4962_2005.docx', '2006-03-06', '2005-12-31', '2005', 'https://www.sec.gov/Archives/edgar/data/4962/000119312506046095/d10k.htm', '0', '7571', '0', '7571', '0', '0', '0', '0', '沒有item 1', '沒有item 7']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4962/000095012305002932/0000950123-05-002932-index.htm
Form 10-K
2004
7284 0 0
0 0
0 0 0
沒有item 1
沒有item 7
['4962', '4962_2004.docx', '2005-03-10', '2004-12-31', '2004', 'https://www.sec.gov/Archives/edgar/data/4962/000095012305002932/y06418e10vk.txt', '7284', '0', '0', '0', '0', '0', '0', '0', '沒有item 1', '沒有item 7']
errnocom []
h <

7950 17259 0
17259 46868
50516 108732 0
['4969', '4969_2017.docx', '2018-03-01', '2017-12-31', '2017', 'https://www.sec.gov/Archives/edgar/data/4969/000000496918000006/d17123110k.htm', '7950', '17259', '0', '17259', '46868', '50516', '108732', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4969/000000496917000006/0000004969-17-000006-index.htm
Form 10-K
2016
7586 16944 0
16944 45171
48471 104986 0
['4969', '4969_2016.docx', '2017-02-27', '2016-12-31', '2016', 'https://www.sec.gov/Archives/edgar/data/4969/000000496917000006/d12312016.htm', '7586', '16944', '0', '16944', '45171', '48471', '104986', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4969/000119312516503634/0001193125-16-503634-index.htm
Form 10-K
2015
0 0 0
0 0
0 0 0
沒

errnocom []
h <div id="formName">
<strong>Form 10-K405</strong> - Annual report [Sections 13 and 15(d), S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4969/000095011701000662/0000950117-01-000662-index.htm
Form 10-K405
2000
3227 0 18166
0 0
20744 33013 0
沒有item 1A，item1結尾到item2
['4969', '4969_2000.docx', '2001-04-02', '2000-12-31', '2000', 'https://www.sec.gov/Archives/edgar/data/4969/000095011701000662/0000950117-01-000662-0001.txt', '3227', '0', '18166', '0', '0', '20744', '33013', '0', '沒有item 1A，item1結尾到item2']
errnocom []
h <div id="formName">
<strong>Form 10-K405</strong> - Annual report [Sections 13 and 15(d), S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4969/0000004969-00-000001-index.html
Form 10-K405
1999
4370 0 19619
0 0
22039 33086 0
沒有item 1A，item1結尾到item2
['4969', '4969_1999.docx', '2000-03-30', '1999-12-31', '1999', 'https://www.sec.gov/Archives/edgar/data/4969/000000496900000001/0000004969-00-000001.txt', '4370', '0', '19619

errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4977/000119312511047639/0001193125-11-047639-index.htm
Form 10-K
2010
3875 3911 0
3911 3953
4357 4471 0
['4977', '4977_2010.docx', '2011-02-25', '2010-12-31', '2010', 'https://www.sec.gov/Archives/edgar/data/4977/000119312511047639/d10k.htm', '3875', '3911', '0', '3911', '3953', '4357', '4471', '0']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4977/000119312510043173/0001193125-10-043173-index.htm
Form 10-K
2009
0 0 0
0 0
0 0 0
沒有item 1
沒有item 7
['4977', '4977_2009.docx', '2010-02-26', '2009-12-31', '2009', 'https://www.sec.gov/Archives/edgar/data/4977/000119312510043173/d10k.htm', '0', '0', '0', '0', '0', '0', '0', '0', '沒有item 1', '沒有item 7']
errnocom []
h <div id="formName">
<strong>Form

6576 0 6654
0 0
7275 0 7427
沒有item 1A，item1結尾到item2
沒有item 7A，item7結尾到item8
['4977', '4977_1994.docx', '1995-03-28', '1994-12-31', '1994', 'https://www.sec.gov/Archives/edgar/data/4977/0000004977-95-000006.txt', '6576', '0', '6654', '0', '0', '7275', '0', '7427', '沒有item 1A，item1結尾到item2', '沒有item 7A，item7結尾到item8']
errnocom []
h <div id="formName">
<strong>Form 10-K</strong> - Annual report [Section 13 and 15(d), not S-K Item 405]: 
      </div>
https://www.sec.gov/Archives/edgar/data/4977/0000004977-94-000006-index.html
Form 10-K
1993
6456 0 6531
0 0
7131 0 7280
沒有item 1A，item1結尾到item2
沒有item 7A，item7結尾到item8
['4977', '4977_1993.docx', '1994-03-30', '1993-12-31', '1993', 'https://www.sec.gov/Archives/edgar/data/4977/0000004977-94-000006.txt', '6456', '0', '6531', '0', '0', '7131', '0', '7280', '沒有item 1A，item1結尾到item2', '沒有item 7A，item7結尾到item8']
errnocom []
************************第 100 筆資料*************************
5009
CompanyName : AMERICAN FILTRONA CORP
CompanyCik : 5009
h <div i

In [ ]:
  

        #import要搜尋的資料
        #輸出filing date等到csv
        #if是10k或是10k405才繼續搜索網址(這裡順序要改)
        #並且是html還是純txt要分開搜尋(如果找不到html才開啟文字模式)
        #找到文字放入datafreame或是直接輸出成docx       
        
        #寫入狀態檔
        do = docx.Document()
        do.add_paragraph(state)
        do.save(newdir+'\\'+str(ccik)+".docx")

In [ ]:
    for k in c:
        temp2=k["href"]
        if(temp2.find("10-k")!=-1):
            docurl.append(temp2)
            break
        elif(temp2.find("10-k")==-1 and temp2.find(".txt")!=-1):
            docurl.append(temp2)
            break
        else:
            continue
        url3='https://www.sec.gov'+temp2
        print(3,url3)